# Diving for Causal Pearls

## Johannes M. Halkenhaeusser
## Minerva Schools at KGI
## IL181.006 – Prof. Scheffler
## Fall 2020



If you have spent enough time around researchers in social sciences, you have at one point heard about DAGs. Most likely you ignored it as just another random acronym relevant to only a subfield of your area of research. However, DAGs or Directed Acyclic Graphs are a powerful way to visualize causal relationships, translate them to/from real structural equation models, use them to derive methods of causal inference, or cast them to linear regressions. But let’s take it one step at a time to go through those features and look at how DAGs can help us think about interventions and calculate the causal effect of coffee on our productivity using inverse probability weighting.

### Coffee three ways
Researchers like to talk about their models, assumptions, and hypothesis in round about ways that can be confusing. But DAGs clear up confusion. If we wanted to test hypothesis that coffee (X) increases the number of words written for an assignment (Y). We suspect that X and Y are affected by the number of sleeping hours we had the day before (Z). Coffee also changes our level of happiness (H). First, we can represent our hypotheses in a DAG in which each variable is denoted by a node and the direction of effect is denoted by the arrows (Fig. 1). 

A second way we can express the relationship by decomposing it into a structural equation model (SEM) consiting of exogenous and endogenous variables that are connected through fucntions. Exogenous variables are those that are not defined by any other variable in the graph meaning they have no arrows pointing toward them (technically they are further defined by another random distribution but we will leave this out for simplicity). Endogenous variables are defined by the exogenous variables through functions F. Fig. 1 can be decomposed into a SEM. Notice how suddenly our DAG lead to some neat linear equations popping out in the functions. They almost look like something we could use in a regression.

$Exogenous:$ {Z}

$Endogenous:$ {X,Y,H}

Functions:
- $f_X:M= \alpha Z$
- $f_Y: Y= \omega X+ \gamma Z$
- $f_H: H = \theta X$




Thirdly, through product decomposition you can express the relationships in the model in terms of the probabilities of one another.That means you can read the joint probability of the variables off the graph by using the product rule: 

$P(x_1,x_2,…,x_n )= \sum_i P(x_i | pa_i) $

where pa_i are the parents (all the nodes pointing into x_i) of the variable. It is easiest to go from exogenous to exogenous variables so we start with laying a probability distribution over the hours of sleep P(Z) and moving multiplying that with the distribution over our coffee intake given the amount of sleep we got P(X | Z). As we make our way throughthe model we end up with: 

$P(X,Y,Z,H)=P(Z)P(X│Z)P(Y│Z,X)P(H|X)$

Decomposing the mode lets us cast our caffeine induced theory in light of probabilities that lets us express uncertainty within the model and depict that in the real world hours of sleep is an integer value but probabily normally distrubted around some average. 
Notice how our research question necessitates making a modeling choice that the coffee affects words written and how the DAG clearly visualises this choice. With coffee this may seem inconsequential but consider a recent paper testing if being part of a minority leads to increased experience of police force (Knox, Lowe, & Mummolo, 2020). In politically charged subject areas like this, creating a DAG that clearly expresses assumptions and the complex interplay of variables allows for more clarity. It also allows us to detect variables that may be confounding our causal relationship or have brought about selection bias. 


### Coffee, Colliders, and Confounders

Selection bias occurs when we control for a variable that is the common effect of two variables that are independent. In statistics lingo, they become conditionally dependent as we condition on a collider.

$ P(X | collider)    \not\!\perp\!\!\!\perp P(Y | collider) $


Image that we are happy (H) whenever it is sunny (S) or we have had coffee and that the number of words written also somehow depends on the amount of sun. However, we consume a sun-independent amount of coffee (Fig. 2). If we were condition on the level of happiness, we introduce a spurious correlation between coffee and the level of sunshine. Usually sun would not tell us anything about our coffee consumption or words written, but because we need coffee or sun to be happy and have information about our happiness, we can make inferences about the X with by knowing S. They have become conditionally dependent through a so-called collider, H. This means that there is a new path between X and Y, that goes through H and S. 
A relationship between two variables is confounded, if the variables have a common cause. In our example, the amount of sleep (Z) we got affects both our treatment (coffee) and outcome (words). In a world where we do not account for the number of sleep we have gotten and there is actually no effect of X on Y, we may attribute the effect sleep has on the words written to coffee and grossly misjudge see our hypothesis as proven through this spurious correlation. 
To deal with colliders and confounders we have to control for confounders (or a variable on their path) and avoid conditioning on colliders and their descendants or block the path by conditioning on a variable in within the path. Once we have accounted for these sources of spurious correlations, we have satisfied the backdoor criteria: 



### Causal Effect Rule
If we now wanted to estimate the causal relationship between coffee and words we can use the causal effect rule: 



Notice the do(X = x) operator in this equation, which signals that we are carrying out an intervention. An intervention occurs when the variable X is forced to take on some value. This fundamentally changes the graph as now it is not the parents of X that determine its value but the intervention. Graphically, we delete all the edges going into X (Fig. 3). Through summing over all the combinations of z, we adjust for them mathematically. We can estimate the causal effect of treatment by calculating the difference between the outcome probability with two different interventions. This is known as the Adjustment Formula:

$ P(Y = 1 | do(X = 1)) - P(Y =1 | do(X = 0)) $

The causal effect rule can also be rewritten as: 

$P (y | do(x)) = \sum_z \frac{P( X = x, Y = y, PA = z)}{P(X = x | PA = z)} $

The denominator P(X=x ┤|  PA=z) is the probability of being treated given the a set of other variables. This is also called the propensity score which we can use in a causal inference method called inverse probability weighting which explicitely makes use of the propensity score.  


### Inverse probability weighting

When we use the regular causal effect rule, we have to sum over each combination of the parent variables of X. Now consider a system that has many parents with many different variables that take many discrete values. The amount of data for each stratum and computational capacity needed quickly skyrockets. Instead we can estimate the propensity score through a logistic or linear regression to determine the likelihood of being treated for each observation. The goal is to balance the treatment probability between the treatment and control groups as to make the effect of the parent influences irrelevant. Inverse probability weighting reweights every observation with its propensity score and adds them up to create a weighted average. Let’s examine it in a toy example of our coffee drinking scenario. 


In [322]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sts
import random as rd
from sklearn import linear_model

In [411]:
n = 10000

#first we randomly choose an exogenous amount of sleep
sleep = sts.randint.rvs(7,9,size =n)

#the probability of drinking coffee is a function of sleep 
coffee_given_sleep = [0.67 if i == 7 else .33 for i in sleep]

#see who is drinking coffee
coffee = np.asarray([1 if sts.uniform.rvs() < i else 0 for i in coffee_given_sleep])

#get the probability of writing any words given coffee drinking and sleeping
#Coffee increases the probability of words written by 0.25 and an extra hour of sleep decreases it by 0.05.
words_given_sleep_coffee = 0.25 * coffee + 0.05*(sleep - 7) + .05*sts.norm.rvs(size =n)

#see who is drinking coffee
words = np.asarray([1 if sts.uniform.rvs() < i else 0 for i in words_given_sleep_coffee])

As shown in the data generating process, sleeping 7 vs 8 hours increases the probability of drinking coffee by 0.34 but less sleep also decreases the probability of sleeping. Coffee increases the probability of writing words by 0.25. This 0.25 is the direct effect of coffee we are interested in. The joint probabilities of each combination of the variables is shown in the table below. Let's see if we can estimate the effect of coffee by reweighting our observations with their appropriate propensity score. 


In [413]:
table = pd.DataFrame({'Sleep':sleep, "Coffee":coffee, "Words" :words, "Real Propensity" :coffee_given_sleep})
joint_p = table.groupby(['Coffee', "Words", "Sleep"]).size().reset_index().rename(columns={0:'Share'})
joint_p.Share /= n 
joint_p.rename(columns={'Share' :"P(X, Y, Z)"}, inplace = True)
joint_p

,Coffee,Words,Sleep,"P(X, Y, Z)"
0,0,0,7,0.1602
1,0,0,8,0.3166
2,0,1,7,0.0037
3,0,1,8,0.0202
4,1,0,7,0.2502
5,1,0,8,0.1135
6,1,1,7,0.0839
7,1,1,8,0.0517


_Table 1: The different combinations of sleeping hours, coffee, and words written and their joint probability_

In [414]:
#Estimate the propensity score using a linear regression model 
p_model = linear_model.LinearRegression()
p_model.fit(sleep.reshape(-1, 1), coffee.reshape(-1, 1))
print("Our estimated linear coefficient is:", p_model.coef_[0,0])


Our estimated linear coefficient is: -0.34179986879790336


This is pretty accurate. Next we estimate the propensity score for the different combinations of our variables.
One could make a weighted average of all the individual observations but we apply the inverse probability to the joint probability as it is faster in this case than reweighing every observation individually. 
We assume/know that sleep and all other exogenous variables are evenly distributed among the groups.

In [415]:

weighted = []

#for each combination of sleepers we weight the different joint probabilities with the propensity score
for i, row in joint_p.iterrows():

    #calculate the propensity score
    p_score = p_model.predict(np.asarray(row['Sleep']).reshape(1, -1))

    #weight the joint probability 
    weighted.append(row['P(X, Y, Z)']/p_score[0,0])

#let's add it to our table
joint_p['Weighted'] = weighted

In [416]:
joint_p

,Coffee,Words,Sleep,"P(X, Y, Z)",Weighted
0,0,0,7,0.1602,0.238790
1,0,0,8,0.3166,0.962065
2,0,1,7,0.0037,0.005515
3,0,1,8,0.0202,0.061383
4,1,0,7,0.2502,0.372941
5,1,0,8,0.1135,0.344897
6,1,1,7,0.0839,0.125059
7,1,1,8,0.0517,0.157103


In [417]:
final_table = pd.pivot_table(joint_p, ['Weighted'], ["Coffee", "Words"],aggfunc = np.sum)
final_table

Weighted
Coffee Words          
0      0      1.200855
       1      0.066898
1      0      0.717838
       1      0.282162

Now using the Adjustment formula

$ P(Y = 1 | do(X = 1)) - P(Y =1 | do(X = 0)) $:

In [410]:
print("Causal Effect of Coffee ~", final_table["Weighted"].values[3] - final_table["Weighted"].values[1])

Causal Effect of Coffee ~ 0.21565125974805055


Obviously there is still some error in the estimation which creeps in from the non-deterministic portion of the model. 